In [1]:
#%% 
from __future__ import print_function, division
import os
import numpy as np
from scipy import linalg
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from smt.utils.misc import compute_rms_error

from smt.problems import Sphere, NdimRobotArm, Rosenbrock
from smt.sampling_methods import LHS
from smt.sampling_methods import Random
from smt.surrogate_models import LS, QP, KPLS, KRG, KPLSK, GEKPLS, MGP


#to ignore warning messages
import warnings
warnings.filterwarnings("ignore")

try:
    from smt.surrogate_models import IDW, RBF, RMTC, RMTB
    compiled_available = True
except:
    compiled_available = False

try:
    import matplotlib.pyplot as plt
    plot_status = True
except:
    plot_status = False

import scipy.interpolate

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
from smt.applications import MOE
from smt.sampling_methods import FullFactorial
import matplotlib.pyplot as plt

#from IPython.display import display, Javascript



In [2]:
# SELECT FUNTION 
# function_1, function_2, function_3 ....

function_ = 'function_3' 

print('selected function :', function_ )

selected function : function_3


In [3]:
#%%
from smt.problems import TensorProduct

#function_ = 'function_7' 
xt = np.load(os.getcwd() + f'\\initial_data\\{function_}\\initial_inputs.npy')
yt = np.load(os.getcwd() + f'\\initial_data\\{function_}\\initial_outputs.npy')
# MOE1: Find the best surrogate model on the whole domain

ndim = xt.shape[1]
functions = {'function_1' : [6.956479380350923*np.exp(-24), 6.956479380350923*np.exp(-24), 1.0849193869919068*np.exp(-19),
                             3.688188175654766*np.exp(-18), 2.2372411166569902*np.exp(-17), 8.958780229862187*np.exp(-47), 4.923470703856277*np.exp(-17),
                             8.218049963770359*np.exp(-23)],
             'function_2' : [0.535473333, 0.3072606735408653, 0.633453646, 0.5235943465204218, 0.6232329336481862, 0.5925262946623997, 0.26613739997551555, 0.37851663725932566],
             'function_3' : [-0.010063223, -0.003617993, -0.108869677098361, -0.036187183, -0.005894685, -0.020458961, -0.121473568, -0.01764569],
             'function_4' : [-0.852032278, -1.724647334, -9.029892788, -2.596945823, -2.780597185, 0.16080956473882368, 0.23857069917150797],
             'function_5' : [197.9373003, 1048.086608259178, 1947.2874420933802, 3284.5243198044022, 3936.987752691248, 5704.044572223114, 4507.8106692894935, 3546.770408355354],
             'function_6' : [-0.765228547, -1.476694517, -0.310302864, -0.382771996, -0.882200783, -0.557927537, -0.791280658, -0.792729805],
             'function_7' : [0.363843663, 0.36384366313112276, 2.4260007022004753, 2.455428195886852, 2.177793073929895, 2.094441487413115, 2.619881006244538, 0.1439275286427469],
             'function_8' : [6.94185601, 6.94185601, 7.651350603604101, 9.037107654, 9.264083347278099, 9.44845039, 9.679689635]         
            };


max_obs = 0
num_queries = len(functions['function_1']) + 1
for i in range(0, num_queries):
    moe1 = MOE(n_clusters=1,variances_support=True, derivatives_support=True, heaviside_optimization=True)
    print("MOE1 enabled experts: ", moe1.enabled_experts)
    moe1.set_training_values(xt, yt)
    moe1.train() 
    
       
    
    prob = TensorProduct(ndim=ndim, func="gaussian")
    xlimits = prob.xlimits
    
    # Fix space
    if(function_ == 'function_1'and i > 1):
        xlimits = np.array([[0.0, 1.0], [0.0, 1.0]])
    if(function_ == 'function_2'and i > 1):
        xlimits = np.array([[0.0, 1.0], [0.0, 1.0]])
    if(function_ == 'function_3'and i > 1):
        xlimits = np.array([[0.0, 1.0], [0.0, 1.0],[0.0, 1.0]])
    if(function_ == 'function_5'and i > 1):
        xlimits = np.array([[0.0, 1.0], [0.0, 1.0],[0.0, 1.0],[0.0, 1.0]])
    if(function_ == 'function_7'and i > 1):
        xlimits = np.array([[0.0, 1.0], [0.0, 1.0],[0.0, 1.0],[0.0, 1.0],[0.0, 1.0],[0.0, 1.0]])
  
    
    if(function_ == 'function_4'and i > 0):
        xlimits = np.array([[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0]])
    if(function_ == 'function_6'and i > 0):
        xlimits = np.array([[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]])    
    if(function_ == 'function_8'and i > -1):
        xlimits = np.array([[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0],[0.0, 1.0]])
    
    if(function_ == 'function_8'and i > 1):
        xlimits = np.array([[0.0, 0.9], [0.0, 0.9],[0.0, 0.9], [0.0, 0.9], [0.0, 0.9],[0.0, 0.9], [0.0, 0.9],[0.0, 0.9]])
        
    if(function_ == 'function_1'and i > 2):
        xlimits = np.array([[0.6, 0.8], [0.6, 0.8]])
    
    if(function_ == 'function_7'and i > 2):
        xlimits = np.array([[0.05, 0.95], [0.05, 0.95],[0.05, 0.95],[0.05, 0.95],[0.05, 0.95],[0.05, 0.95]])
    
    if(function_ == 'function_8'and i > 2):
        xlimits = np.array([[0.1, 0.9], [0.1, 0.9],[0.1, 0.9],[0.1, 0.9],[0.1, 0.9],[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]])
    
    if(function_ == 'function_1'and i > 3):
        xlimits = np.array([[0.68, 0.79], [0.68, 0.79]])    
    if(function_ == 'function_8'and i > 3):
        xlimits = np.array([[0.2, 0.8], [0.1, 0.9],[0.2, 0.8],[0.1, 0.9],[0.1, 0.9],[0.1, 0.75], [0.45, 0.95], [0.01, 0.99]])
    
    if(function_ == 'function_1'and i > 4):
        xlimits = np.array([[0.1, 0.9], [0.1, 0.9]])
    
    if(function_ == 'function_1'and i > 6):
        xlimits = np.array([[0.62, 1.0], [0.62, 1.0]])
    if(function_ == 'function_4'and i > 6):
        xlimits = np.array([[0.2, 0.58], [0.2, 0.58], [0.2, 0.58], [0.2, 0.58]])
    if(function_ == 'function_8'and i > 6):
        xlimits = np.array([[0.01, 0.99], [0.01, 0.99],[0.01, 1.0],[0.1, 0.9],[0.1, 0.9],[0.1, 0.9], [0.45, 0.99], [0.01, 0.99]])
    

         
    samp = LHS(xlimits=xlimits, random_state=42, criterion='correlation' )
    x = samp(10000)
    y_moe1 = moe1.predict_values(x)
    '''
    ucb = y_moe1 + 1.96 * np.sqrt(moe1.predict_variances(x))
    idx_max = np.argmax(ucb)
    next_query = x[idx_max]
    print('Next query UCB: ', next_query)
    #res = pred - 3.0 * np.sqrt(var)
    '''
    
    acquisition_function = []
    Z = (y_moe1 - max_obs - 0) / np.sqrt(moe1.predict_variances(x))
    probs = scipy.stats.norm.cdf(Z)
    acquisition_function = np.array(probs)
    
    idx_max = np.argmax(acquisition_function)
    next_query = x[idx_max]
    
    if i < len(functions['function_1']):
        #xt = np.append(xt, np.array(next_query))
        xt = np.concatenate((xt, np.array([next_query])))
        yt = np.append(yt, functions[function_][i])
        
        max_obs = max(max_obs, functions[function_][i])
    print('------------------>>>>>>Next query EI: ', next_query) 
  

MOE1 enabled experts:  ['KRG', 'KPLS', 'KPLSK']
Kriging 0.0261192753127073
KPLS 0.020519089963858345
KPLSK 0.026119207249466085
Best expert = KPLS
------------------>>>>>>Next query EI:  [0.43688468 0.45306649 0.44261551]
MOE1 enabled experts:  ['KRG', 'KPLS', 'KPLSK']
Kriging 0.013144107921350606
KPLS 0.02209326055660214
KPLSK 0.03268768523847145
Best expert = Kriging
------------------>>>>>>Next query EI:  [0.5133934  0.61171533 0.41879058]
MOE1 enabled experts:  ['KRG', 'KPLS', 'KPLSK']
Kriging 0.005550204158906664
KPLS 0.026004966953546656
KPLSK 0.03028482131779679
Best expert = Kriging
------------------>>>>>>Next query EI:  [0.99740714 0.48776558 0.85230696]
MOE1 enabled experts:  ['KRG', 'KPLS', 'KPLSK']
Kriging 0.03766087712158942
KPLS 0.021932955906713396
KPLSK 0.037660891274354624
Best expert = KPLS
------------------>>>>>>Next query EI:  [0.43043884 0.60462141 0.43822484]
MOE1 enabled experts:  ['KRG', 'KPLS', 'KPLSK']
Kriging 0.04002307445340763
KPLS 0.031614953558326386
KP